In [1]:
# =============================================================
# 🚦 PEMS-BAY Traffic Forecasting
# MODEL: Deep GraphWaveNet (Paper-style STGNN)
# =============================================================


# =============================================================
# 0️⃣ IMPORTS + DEVICE + STABILITY SETTINGS
# =============================================================
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import random


# -------------------------------------------------------------
# Reproducibility (VERY IMPORTANT for projects/research)
# -------------------------------------------------------------
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


# -------------------------------------------------------------
# Device
# -------------------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


# -------------------------------------------------------------
# GPU speed boost (safe)
# -------------------------------------------------------------
torch.backends.cudnn.benchmark = True


Device: cuda


In [2]:
# =============================================================
# 1️⃣ PATHS (SAFE + PORTABLE VERSION)
# =============================================================
import os

BASE_DIR = os.getcwd()   # current working directory

csv_path = os.path.join(BASE_DIR, "pems_bay_final_with_extra_features.csv")
adj_path = os.path.join(BASE_DIR, "adj_mx_PEMS-BAY.pkl")


# -------------------------------------------------------------
# Safety check (prevents silent file errors)
# -------------------------------------------------------------
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"CSV not found: {csv_path}")

if not os.path.exists(adj_path):
    raise FileNotFoundError(f"Adjacency file not found: {adj_path}")


print("CSV Path:", csv_path)
print("Adj Path:", adj_path)


CSV Path: C:\Users\akanksh_02\Downloads\trf\pems_bay_final_with_extra_features.csv
Adj Path: C:\Users\akanksh_02\Downloads\trf\adj_mx_PEMS-BAY.pkl


In [3]:
# =============================================================
# 2️⃣ LOAD CSV (MEMORY OPTIMIZED)
# =============================================================
print("\n📂 Loading CSV...")

df = pd.read_csv(
    csv_path,
    index_col="timestamp",
    parse_dates=True,
    low_memory=False
)

print("Dataset shape:", df.shape)
print("Columns:", len(df.columns))


# -------------------------------------------------------------
# Convert numeric columns → float32 (50% memory reduction)
# -------------------------------------------------------------
for col in df.columns:
    if df[col].dtype == "float64":
        df[col] = df[col].astype("float32")

print("Memory optimized ✓")



📂 Loading CSV...
Dataset shape: (52116, 338)
Columns: 338
Memory optimized ✓


In [4]:
# =============================================================
# 3️⃣ SELECT COLUMNS (SAFE + MEMORY OPTIMIZED)
# =============================================================

print("\n🧩 Selecting sensor + time features...")

# -------------------------------------------------------------
# Sensor columns (graph nodes)
# -------------------------------------------------------------
sensor_cols = [c for c in df.columns if c.isdigit()]

if len(sensor_cols) == 0:
    raise ValueError("No sensor columns detected!")

print("Number of sensors (nodes):", len(sensor_cols))


# -------------------------------------------------------------
# Time features (extra node features)
# -------------------------------------------------------------
time_cols = [
    "hour_sin", "hour_cos",
    "dow_sin", "dow_cos",
    "weekend", "holiday"
]

for c in time_cols:
    if c not in df.columns:
        raise ValueError(f"Missing time feature column: {c}")


# -------------------------------------------------------------
# Convert to numpy float32 (IMPORTANT)
# -------------------------------------------------------------
traffic = df[sensor_cols].to_numpy(dtype=np.float32)
time_feat = df[time_cols].to_numpy(dtype=np.float32)


print("Traffic shape     :", traffic.shape)   # (T, N)
print("Time feat shape   :", time_feat.shape) # (T, F_time)



🧩 Selecting sensor + time features...
Number of sensors (nodes): 325
Traffic shape     : (52116, 325)
Time feat shape   : (52116, 6)


In [5]:
# =============================================================
# 4️⃣ NORMALIZE TRAFFIC (PER-SENSOR NORMALIZATION ⭐)
# =============================================================

print("\n📊 Normalizing traffic per sensor...")

# compute mean/std for EACH sensor (column-wise)
mean = traffic.mean(axis=0, keepdims=True)
std  = traffic.std(axis=0, keepdims=True)

# avoid divide by zero
std[std == 0] = 1.0

traffic = (traffic - mean) / std

traffic = traffic.astype(np.float32)

print("Normalized ✓")
print("Mean shape:", mean.shape)  # (1, N)
print("Std shape :", std.shape)



📊 Normalizing traffic per sensor...
Normalized ✓
Mean shape: (1, 325)
Std shape : (1, 325)


In [6]:
# =============================================================
# 5️⃣ LOAD + NORMALIZE ADJACENCY (RESEARCH STANDARD ⭐)
# =============================================================

print("\n🕸 Loading adjacency...")

with open(adj_path, "rb") as f:
    adj_data = pickle.load(f, encoding="latin1")

A = adj_data[2].astype(np.float32)

print("Raw adjacency:", A.shape)


# -------------------------------------------------------------
# Add self-loops
# -------------------------------------------------------------
A = A + np.eye(A.shape[0], dtype=np.float32)


# -------------------------------------------------------------
# Symmetric normalization: D^-1/2 A D^-1/2
# -------------------------------------------------------------
D = np.sum(A, axis=1)
D_inv_sqrt = np.diag(1.0 / np.sqrt(D + 1e-8))

A_norm = D_inv_sqrt @ A @ D_inv_sqrt


# -------------------------------------------------------------
# Convert to torch
# -------------------------------------------------------------
adj_mx = torch.tensor(A_norm, dtype=torch.float32).to(device)

print("Normalized adjacency shape:", adj_mx.shape)



🕸 Loading adjacency...
Raw adjacency: (325, 325)
Normalized adjacency shape: torch.Size([325, 325])


In [7]:
# =============================================================
# 6️⃣ ADD TIME FEATURES TO EVERY NODE (MEMORY SAFE ⭐)
# =============================================================

print("\n🔗 Combining traffic + time features...")

T, N = traffic.shape
F_time = time_feat.shape[1]

# -------------------------------------------------------------
# Expand dims safely (no heavy copy)
# -------------------------------------------------------------
traffic = traffic[..., None]          # (T, N, 1)

# broadcast instead of repeat (VERY IMPORTANT)
time_feat_expanded = np.broadcast_to(
    time_feat[:, None, :],            # (T,1,F_time)
    (T, N, F_time)                   # (T,N,F_time)
)

# -------------------------------------------------------------
# Concatenate features
# -------------------------------------------------------------
data = np.concatenate(
    [traffic, time_feat_expanded],
    axis=2
).astype(np.float32)


print("Time steps (T):", T)
print("Nodes (N):", N)
print("Features per node:", data.shape[2])
print("Final data shape:", data.shape)



🔗 Combining traffic + time features...
Time steps (T): 52116
Nodes (N): 325
Features per node: 7
Final data shape: (52116, 325, 7)


In [8]:
# =============================================================
# 7️⃣ MEMORY SAFE DATASET (PROFESSIONAL VERSION ⭐)
# =============================================================

SEQ_LEN = 24
PRED_LEN = 3


class TrafficDataset(Dataset):

    def __init__(self, data):
        # ⭐ keep as numpy (NO big torch copy)
        self.data = data.astype(np.float32)

    def __len__(self):
        return len(self.data) - SEQ_LEN - PRED_LEN

    def __getitem__(self, idx):

        # -----------------------------------------------------
        # slice windows (numpy)
        # -----------------------------------------------------
        x = self.data[idx : idx+SEQ_LEN]                 # (T,N,F)
        y = self.data[idx+SEQ_LEN : idx+SEQ_LEN+PRED_LEN, :, 0]  # (P,N)

        # -----------------------------------------------------
        # convert ONLY this sample to torch (fast)
        # -----------------------------------------------------
        x = torch.from_numpy(x).permute(2,1,0)  # (F,N,T)
        y = torch.from_numpy(y)

        return x, y


In [9]:
# =============================================================
# 8️⃣ TRAIN / TEST SPLIT + DATALOADER (FINAL SAFE VERSION ⭐)
# =============================================================

print("\n📦 Creating train/test split...")

split = int(len(data) * 0.8)

train_data = data[:split]
test_data  = data[split:]

print("Train samples:", len(train_data))
print("Test samples :", len(test_data))


BATCH_SIZE = 64


# -------------------------------------------------------------
# Windows + CUDA safe loaders
# -------------------------------------------------------------
train_loader = DataLoader(
    TrafficDataset(train_data),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,      # ⭐ Windows safe
    pin_memory=True,    # ⭐ faster GPU transfer
    drop_last=True      # ⭐ stable batches
)

test_loader = DataLoader(
    TrafficDataset(test_data),
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

print("Batches per epoch:", len(train_loader))



📦 Creating train/test split...
Train samples: 41692
Test samples : 10424
Batches per epoch: 651


In [10]:
# =============================================================
# STEP 2️⃣  MTGNN MODEL (ONLY REPLACE MODEL PART)
# =============================================================

import torch
import torch.nn as nn
import torch.nn.functional as F


class MTGNN(nn.Module):
    """
    MTGNN – Multivariate Time-series Graph Neural Network
    Paper: Connecting the Dots (2020)

    Key ideas:
    - Adaptive Graph Learning
    - Temporal CNN (no GRU)
    - Mix-hop Graph Convolution
    - Gated blocks
    """

    def __init__(self, num_nodes, in_dim, out_dim, seq_len):

        super().__init__()

        self.num_nodes = num_nodes
        self.seq_len = seq_len

        channels = 64
        layers = 6

        # =====================================================
        # ⭐ Adaptive graph learning (MOST IMPORTANT PART)
        # Learns graph automatically (beats fixed adjacency)
        # =====================================================
        self.nodevec1 = nn.Parameter(torch.randn(num_nodes, 10))
        self.nodevec2 = nn.Parameter(torch.randn(10, num_nodes))


        # =====================================================
        # Input projection
        # =====================================================
        self.start_conv = nn.Conv2d(in_dim, channels, kernel_size=(1,1))


        # =====================================================
        # Temporal + Graph blocks
        # =====================================================
        self.temporal_convs = nn.ModuleList()
        self.graph_convs = nn.ModuleList()

        for _ in range(layers):

            # Temporal CNN
            self.temporal_convs.append(
                nn.Conv2d(
                    channels,
                    channels,
                    kernel_size=(1,3),
                    padding=(0,1)
                )
            )

            # Graph mixing
            self.graph_convs.append(
                nn.Linear(num_nodes, num_nodes, bias=False)
            )


        # =====================================================
        # Output head
        # =====================================================
        self.end_conv = nn.Conv2d(channels, out_dim, kernel_size=(1,1))


    # =========================================================
    # Adaptive graph creation
    # =========================================================
    def get_adj(self):
        adj = F.relu(torch.mm(self.nodevec1, self.nodevec2))
        adj = F.softmax(adj, dim=1)
        return adj


    # =========================================================
    # Forward
    # =========================================================
    def forward(self, x):

        adj = self.get_adj()

        x = self.start_conv(x)

        for tconv, gconv in zip(self.temporal_convs, self.graph_convs):

            residual = x

            # temporal
            x = F.relu(tconv(x))

            # graph
            x = torch.einsum("bfnt,nm->bfmt", x, adj)

            x = x + residual

        x = self.end_conv(x)

        return x.mean(dim=-1)


In [11]:
# =============================================================
# STEP 3️⃣  MTGNN INITIALIZATION
# =============================================================

print("\n🧠 Initializing MTGNN model...")

model = MTGNN(
    num_nodes=N,            # 325 sensors
    in_dim=data.shape[2],   # 7 features
    out_dim=PRED_LEN,       # predict 3 steps
    seq_len=SEQ_LEN         # 24 history
).to(device)


total_params = sum(p.numel() for p in model.parameters())

print("Input features :", data.shape[2])
print("Nodes          :", N)
print("Seq length     :", SEQ_LEN)
print("Prediction     :", PRED_LEN)
print("Parameters     :", round(total_params/1e6, 2), "M")
print("Device         :", device)
print("MTGNN ready ✓")



🧠 Initializing MTGNN model...
Input features : 7
Nodes          : 325
Seq length     : 24
Prediction     : 3
Parameters     : 0.72 M
Device         : cuda
MTGNN ready ✓


In [12]:
# =============================================================
# STEP 4️⃣  MTGNN TRAINING LOOP (FAST + STABLE)
# =============================================================

from tqdm import tqdm

criterion = nn.L1Loss()          # MAE (best for traffic)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

EPOCHS = 25


# ⭐ MTGNN can use larger batch (faster than GraphWaveNet)
train_loader = DataLoader(
    TrafficDataset(train_data),
    batch_size=64,          # bigger batch → faster GPU
    shuffle=True,
    num_workers=0,          # Windows safe
    pin_memory=True
)


print("\n🚀 Training MTGNN...\n")


for epoch in range(EPOCHS):

    model.train()
    epoch_loss = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")

    for x, y in pbar:

        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        optimizer.zero_grad()

        pred = model(x)

        loss = criterion(pred, y)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)  # ⭐ stability

        optimizer.step()

        epoch_loss += loss.item()

        pbar.set_postfix(loss=loss.item())


    print(f"✅ Epoch {epoch+1}/{EPOCHS}  Loss: {epoch_loss/len(train_loader):.4f}\n")



🚀 Training MTGNN...



Epoch 1/25: 100%|█████████████████████████████████████████████████████████| 652/652 [03:34<00:00,  3.04it/s, loss=0.44]


✅ Epoch 1/25  Loss: 0.3360



Epoch 2/25: 100%|████████████████████████████████████████████████████████| 652/652 [03:33<00:00,  3.05it/s, loss=0.682]


✅ Epoch 2/25  Loss: 0.2837



...

✅ Epoch 23/25  Loss: 0.1736



Epoch 24/25: 100%|███████████████████████████████████████████████████████| 652/652 [03:48<00:00,  2.85it/s, loss=0.105]


✅ Epoch 24/25  Loss: 0.1723



Epoch 25/25: 100%|███████████████████████████████████████████████████████| 652/652 [03:53<00:00,  2.79it/s, loss=0.225]

✅ Epoch 25/25  Loss: 0.1709



In [16]:
# =============================================================
# FINAL EVALUATION — MTGNN ONLY (FIXED VERSION)
# =============================================================

print("\n📊 Evaluating MTGNN on test set...\n")

model.eval()

mae_sum = 0.0
mse_sum = 0.0
count = 0

all_preds = []
all_true = []

with torch.no_grad():

    for x, y in test_loader:

        x = x.to(device)
        y = y.to(device)

        pred = model(x)

        error = pred - y

        mae_sum += torch.abs(error).sum().item()
        mse_sum += (error ** 2).sum().item()
        count += y.numel()

        all_preds.append(pred.detach().cpu())
        all_true.append(y.detach().cpu())


# =============================================================
# Normalized metrics
# =============================================================
mae_norm = mae_sum / count
rmse_norm = (mse_sum / count) ** 0.5


# =============================================================
# Convert back to REAL scale (FIXED HERE)
# =============================================================
std_scalar = float(np.mean(std))   # ⭐ FIX: convert to scalar

real_mae = mae_norm * std_scalar
real_rmse = rmse_norm * std_scalar


# =============================================================
# R² score
# =============================================================
from sklearn.metrics import r2_score

preds = torch.cat(all_preds).numpy().ravel()
trues = torch.cat(all_true).numpy().ravel()

r2 = r2_score(trues, preds)


# =============================================================
# Print results
# =============================================================
print("===================================")
print(f"Normalized MAE : {mae_norm:.4f}")
print(f"Normalized RMSE: {rmse_norm:.4f}")
print("-----------------------------------")
print(f"Real MAE : {real_mae:.3f}")
print(f"Real RMSE: {real_rmse:.3f}")
print(f"R² score : {r2:.4f}")
print("===================================")


# =============================================================
# Save model
# =============================================================
torch.save(model.state_dict(), "mtgnn_model.pth")
print("Model saved ✓")



📊 Evaluating MTGNN on test set...

Normalized MAE : 0.1784
Normalized RMSE: 0.3884
-----------------------------------
Real MAE : 1.526
Real RMSE: 3.323
R² score : 0.8516
Model saved ✓
